In [1]:
# Install core LangChain package
!pip install langchain==0.1.12

# Install OpenAI integration for LangChain
!pip install langchain-openai==0.0.8

# Install other helpful packages
!pip install python-dotenv configparser

  Using cached langchain-0.1.12-py3-none-any.whl.metadata (13 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached orjson-3.10.18-cp312-cp312-win_amd64.whl.metadata (43 kB)
Using cached langchain-0.1.12-py3-none-any.whl (809 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
Using cached langchain_text_splitters-0.0.2

In [2]:
# Install Google Gemini integration for Langchain
!pip install langchain-google-genai==0.0.11

  Using cached langchain_google_genai-0.0.11-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_generativeai-0.4.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached google_auth-2.40.2-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached grpcio_status-1.71.0-py3-none-any.whl.metadata (1.1 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.70.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.69.0-py

# Getting API Credentials

In [3]:
import locale
import os
import configparser

# Fix  encoding issues that might arise in different environments
locale.getpreferredencoding = lambda: "UTF-8"

# Function to load API keys from config file
def load_api_keys(config_file='config.ini'):
    config = configparser.ConfigParser()
    config.read(config_file)

    api_creds={}

    # Extract OpenAI Key
    if 'openai' in config and 'api_key' in config['openai']:
        api_creds['openai_key']= config['openai']['api_key']

    # If using Gemini, you can extract its key or set it manually
    # Here, for example we'll create a placeholder

    api_creds['gemini_key']=config['gemini']['api_key']

    return api_creds

# Try to load from config.ini, fall back to creating a dict for manual entry

try:
    api_creds = load_api_keys()
    print(f"AVailable API Credentials: {', '.join([k.replace('_key','') for k in api_creds if api_creds[k]])}")

except Exception as e:
    print(f"Error loading config: {e}")
    print("Please add your API keys manually in the next cell.")
    api_creds = {'openai_key': '', 'gemini_key': ''}

AVailable API Credentials: openai, gemini


In [4]:
# Set environment variables for the APIs
os.environ['OPENAI_API_KEY'] = api_creds.get('openai_key', '')
os.environ['GOOGLE_API_KEY'] = api_creds.get('gemini_key', '')

# Intializing OpenAI

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Intialize OpenAI model
# - model_name: Specifies which OpenAi model to use (gpt-3.5-turbo is a good default)
# - temperature: Controls randomness (0.0 = deterministic)
model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.0)

# A prompt Template

In [6]:
# Create a prompt template with a variable {topic}

PROMPT = "Tell me a joke about {topic}"
prompt = ChatPromptTemplate.from_template(PROMPT)

# Create a chain by connecting the prompt template to the language model
# The | operator in Langchain is used to connect components
chain = prompt | model

# Use the chain with a specific topic
response = chain.invoke({"topic": "guns"})

# Display the response
print(response.content)

Why did the gun break up with the bullet? 

Because it just couldn't handle the recoil-tionship!


# Gemini Model Intializtaion

In [7]:
try:
    from langchain_google_genai import ChatGoogleGenerativeAI

    # Intialize Gemini model
    # convert_system_message_to_human = True helps Gemini understand system prompts
    gemini_model = ChatGoogleGenerativeAI(
        model = "gemini-1.5-flash", # Updated to the current model name
        convert_system_message_to_human = True
    )
    print("Google Gemini Model initialized successfully!")

except Exception as e:
    print(f"Could not initialize Gemini model: {e}")
    print("Continuing with OpenAI only.")

Google Gemini Model initialized successfully!


# Prompt Template for Gemini

In [8]:
from langchain.prompts import ChatPromptTemplate

try: 
    # Define the prompt template
    PROMPT = "Tell me a joke about {topic}"
    prompt = ChatPromptTemplate.from_template(PROMPT)

    # Chain the prompt with Gemini model
    gemini_chain = prompt | model

    # Generate a joke using Gemini
    gemini_response = gemini_chain.invoke({"topic":"statistics"})
    print(gemini_response.content)

except Exception as e:
    print(f"Gemini model not available or failed to respond. Error: {e}")

Why did the statistician go to the beach?

To test the waters!


# Multiple topics through Map

In [13]:
# Define multiple topics for jokes

topics = [
    {"topic" : "Anime"},
    {"topic" : "Statistics"},
    {"topic" : "Python"}
]
# Process all topics in parallel using map
# This is much more efficient than running them one by one in a lopp
responses = chain.batch(topics)

# Display all jokes
for i, response in enumerate(responses):
    print(f"Joke about {topics[i]['topic']}:")
    print(response.content)
    print("-----\n")

Joke about Anime:
Why did the anime character go to therapy? 

Because he had too many issues to "resolve"!
-----

Joke about Statistics:
Why did the statistician go to the beach?

To test the waters!
-----

Joke about Python:
Why did the Python programmer get lost in the jungle?

Because he couldn't find his way out of the loop!
-----



# Amnesia of langchain without memory

In [19]:
# Memory Issuse
# Create a very simple prompt template that just passes through the query
prompt = ChatPromptTemplate.from_template("{query}")
basic_chain = prompt | model

# First question about rainbow colors
response = basic_chain.invoke({"query": "What are the first four colors of the rainbow ?"})
print("Query: What are the first four colors of the rainbow ?")
print(f"Response: {response.content}\n")

# Follow up question about the remaining colors
response = basic_chain.invoke({"query": "And the other three ?"})
print("Query: And the other three ?")
print(f"Response: {response.content}\n")
print("Note: The model doesn't remember our previous question about rainbow colors.")

Query: What are the first four colors of the rainbow ?
Response: The first four colors of the rainbow are red, orange, yellow, and green.

Query: And the other three ?
Response: Apologies for the oversight. The other three major crime categories are property crime, violent crime, and drug-related crime. These categories cover a wide range of offenses including theft, assault, robbery, and drug trafficking. Each of these categories is further broken down into specific offenses and committed acts.

Note: The model doesn't remember our previous question about rainbow colors.


# Giving memory to Langchain

In [35]:
# Giving memory to langchain

from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Create a prompt template that includes conversation history
prompt = ChatPromptTemplate.from_messages([
    # System message tells the model how to behave
    ("system","Act as a helpful AI Assistant"),
    # This placeholder will be filled with conversatiin history
    MessagesPlaceholder(variable_name = "history"),
    # This is the user's current input
    ("human","{input}"),
])

# Initialize memory to store conversation history
# k=3 means it will remember the last 3 exchanges () 3 user messages and 3 AI responses)
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [36]:
# Check what's in memory initially (should be empty)
memory.load_memory_variables({})

{'history': []}

In [37]:
# Create the conversation chain with memory
chain=(
    # Step1 : Load the conversation history form memory
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    # Step2 : Format the prompt with history and current input
    | prompt
    # Step3 : Send to the language model
    | model
)

In [38]:
# First question about rainbow colors
user_input={"input" : "What are the first four colors of the rainbow ?"}
response = chain.invoke(user_input)
print(f"User: {user_input['input']}")
print(f"AI: {response.content}")

User: What are the first four colors of the rainbow ?
AI: The first four colors of the rainbow are red, orange, yellow, and green.


In [39]:
# Check memory - still empty because we haven't saved the conversation yet
memory.load_memory_variables({})

{'history': []}

In [40]:
# Important : Save the conversation to memory
memory.save_context(user_input, {"output": response.content})

# Now check what's in memory
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of the rainbow ?'),
  AIMessage(content='The first four colors of the rainbow are red, orange, yellow, and green.')]}

In [41]:
# Ask a follow-up question
user_input = {"input": "And the last 3 ?"}
response = chain.invoke(user_input)
print(f"User: {user_input['input']}")
print(f"AI: {response.content}")

# Save this turn to memory too
memory.save_context(user_input, {"output": response.content})

# Check updated memory
memory.load_memory_variables({})

User: And the last 3 ?
AI: The last three colors of the rainbow are blue, indigo, and violet.


{'history': [HumanMessage(content='What are the first four colors of the rainbow ?'),
  AIMessage(content='The first four colors of the rainbow are red, orange, yellow, and green.'),
  HumanMessage(content='And the last 3 ?'),
  AIMessage(content='The last three colors of the rainbow are blue, indigo, and violet.')]}

# Creating a ChatBot

In [49]:
def run_chatbot(system_prompt='',history_window=3, temperature=1.0, llm=model):
    # Use default system prompt if none provided
    if not system_prompt:
        system_prompt = "Act as a helpful AI Assistant"

    # Create the prompt template
    prompt = ChatPromptTemplate.from_messages([
        # System message tells the model how to behave
        ("system",system_prompt),
        # This placeholder will be filled with conversatiin history
        MessagesPlaceholder(variable_name = "history"),
        # This is the user's current input
        ("human","{input}"),
    ])

    # Create memory
    memory = ConversationBufferWindowMemory(k=history_window, return_messages=True)

    # Create a conversation chaun
    conversation_chain=(
        # Step1 : Load the conversation history form memory
        RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
        )
        # Step2 : Format the prompt with history and current input
        | prompt
        # Step3 : Send to the language model
        | llm
    )

    # Welcome message
    print("Hello! I am your friendly chatbot. Let's Chat! (type 'STOP' to end)")

    # Chat loop
    while True:
        # Get user input
        user_prompt = input('User: >>>')

        # Check if user wants to exit
        if user_prompt.strip().upper() == 'STOP':
            print("Chatbot: >>> Goodbye!")
            break

        # Generate and print the chatbot's reply
        user_inp = {'input': user_prompt}
        reply = conversation_chain.invoke(user_inp)
        print(f"Chatbot: >>>\n{reply.content}")

        # Save to memory
        memory.save_context(user_inp, {"output": reply.content})

In [50]:
# Run the chatbot with default settings
run_chatbot()

Hello! I am your friendly chatbot. Let's Chat! (type 'STOP' to end)


User: >>> books to read


Chatbot: >>>
Sure! Here are some popular books across genres that you may enjoy:

1. Fiction:
   - "To Kill a Mockingbird" by Harper Lee
   - "1984" by George Orwell
   - "The Great Gatsby" by F. Scott Fitzgerald
   - "Pride and Prejudice" by Jane Austen
   - "The Kite Runner" by Khaled Hosseini

2. Mystery/Thriller:
   - "Gone Girl" by Gillian Flynn
   - "The Girl with the Dragon Tattoo" by Stieg Larsson
   - "The Da Vinci Code" by Dan Brown
   - "The Silent Patient" by Alex Michaelides
   - "The Woman in the Window" by A.J. Finn

3. Science Fiction/Fantasy:
   - "Dune" by Frank Herbert
   - "Harry Potter and the Sorcerer's Stone" by J.K. Rowling
   - "The Hobbit" by J.R.R. Tolkien
   - "The Martian" by Andy Weir
   - "Ender's Game" by Orson Scott Card

4. Non-Fiction:
   - "Becoming" by Michelle Obama
   - "Sapiens: A Brief History of Humankind" by Yuval Noah Harari
   - "Educated" by Tara Westover
   - "The Power of Habit" by Charles Duhigg
   - "The Immortal Life of Henrietta Lacks

User: >>> Suggest books similar to 'The seven husbands of Evelyn Hugo'


Chatbot: >>>
If you enjoyed "The Seven Husbands of Evelyn Hugo" by Taylor Jenkins Reid, you might like these books that share similar themes of Hollywood glamour, complex characters, and emotional depth:

1. "Daisy Jones & The Six" by Taylor Jenkins Reid - Another book by the same author that explores the music industry in the 1970s through the intertwined stories of a rock band and its enigmatic lead singer, Daisy Jones.

2. "The Seven or Eight Deaths of Stella Fortuna" by Juliet Grames - This multigenerational family saga follows the life of Stella Fortuna, an Italian immigrant living in America, and the events that shape her extraordinary life.

3. "The Swans of Fifth Avenue" by Melanie Benjamin - A novel inspired by the friendship between author Truman Capote and socialite Babe Paley, exploring the glamour and intrigue of New York's high society in the 1950s and 1960s.

4. "The Seven Husbands of Evelyn Hugo" by Anthony Doerr - A story of love, sacrifice, and survival set against th

...

Chatbot: >>>
I'm sorry, I can't sing a Tamil song as I'm just a text-based AI assistant. However, I can provide you with the lyrics to a Tamil song if you'd like. Just let me know the name of the song or any specific song lyrics you're interested in. I'm here to help with that!


User: >>> yes, give me lyrics of a tamil movie song


Chatbot: >>>
Of course! Here are the lyrics to the famous Tamil movie song "Aalaporan Tamizhan" from the movie Mersal:

Aalaporan Tamizhan
Arasangal Veeshumpothu
Ullaththai Vidaatha Ulakam
Athanaal Solla

Unthan Thamizh Thandhai Neeye
Thandhaikku Nee Irukkum Anbe
Naangal Ketta Oru Kural Thaane
Oorukke Varum Anbe

Thamizhan Endru Sollada
Thalai Nimirndhu Nillada

I hope you enjoy these lyrics from the song "Aalaporan Tamizhan"! If you need more lyrics or information, feel free to ask.


User: >>> stop


Chatbot: >>> Goodbye!
